<a href="https://colab.research.google.com/github/sayakpaul/BERT-for-Mobile/blob/master/Evaluation_SST_2_DistilBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -q https://github.com/sayakpaul/BERT-for-Mobile/releases/download/v0.2.0/SST_Models.zip
!unzip -qq SST_Models.zip

In [2]:
!pip install -q transformers

     |████████████████████████████████| 1.1MB 4.9MB/s 
     |████████████████████████████████| 3.0MB 17.5MB/s 
     |████████████████████████████████| 1.1MB 44.0MB/s 
     |████████████████████████████████| 890kB 30.3MB/s 


In [1]:
import tensorflow as tf
print(tf.__version__)

2.3.0


In [2]:
import os
import numpy as np
import pandas as pd
import transformers

from sklearn.metrics import accuracy_score

# fix random seed for reproducibility
seed = 42
np.random.seed(seed)
tf.random.set_seed(seed)

In [3]:
data_dir = tf.keras.utils.get_file(
      fname='SST-2.zip',
      origin='https://firebasestorage.googleapis.com/v0/b/mtl-sentence-representations.appspot.com/o/data%2FSST-2.zip?alt=media&token=aabc5f6b-e466-44a2-b9b4-cf6337f84ac8',
      extract=True)
data_dir = os.path.join(os.path.dirname(data_dir), 'SST-2')

In [4]:
# The validation set has got labels
valid = os.path.join(data_dir, "dev.tsv")

In [5]:
valid_dataset = pd.read_csv(valid, sep='\t')
valid_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 872 entries, 0 to 871
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sentence  872 non-null    object
 1   label     872 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 13.8+ KB


In [6]:
valid_reviews = valid_dataset['sentence'].values
valid_sentiments = valid_dataset['label'].values

In [7]:
tokenizer = transformers.DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

In [8]:
import tqdm

def create_bert_input_features(tokenizer, docs, max_seq_length):
    
    all_ids, all_masks = [], []
    for doc in tqdm.tqdm(docs, desc="Converting docs to features"):
        tokens = tokenizer.tokenize(doc)
        if len(tokens) > max_seq_length-2:
            tokens = tokens[0 : (max_seq_length-2)]
        tokens = ['[CLS]'] + tokens + ['[SEP]']
        ids = tokenizer.convert_tokens_to_ids(tokens)
        masks = [1] * len(ids)
        # Zero-pad up to the sequence length.
        while len(ids) < max_seq_length:
            ids.append(0)
            masks.append(0)
        all_ids.append(ids)
        all_masks.append(masks)
    encoded = np.array([all_ids, all_masks])
    return encoded

In [9]:
MAX_SEQ_LENGTH = 500

val_features_ids, val_features_masks = create_bert_input_features(tokenizer, valid_reviews, 
                                                                  max_seq_length=MAX_SEQ_LENGTH)
print('Val Features:', val_features_ids.shape, val_features_masks.shape)

Converting docs to features: 100%|██████████| 872/872 [00:00<00:00, 2190.07it/s]

Val Features: (872, 500) (872, 500)


In [10]:
interpreter = tf.lite.Interpreter(model_path="distilbert_sst_tflite.tflite")
interpreter.allocate_tensors()
interpreter.get_input_details()

[{'dtype': numpy.int32,
  'index': 0,
  'name': 'bert_input_ids',
  'quantization': (0.0, 0),
  'quantization_parameters': {'quantized_dimension': 0,
   'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32)},
  'shape': array([  1, 500], dtype=int32),
  'shape_signature': array([  1, 500], dtype=int32),
  'sparsity_parameters': {}},
 {'dtype': numpy.int32,
  'index': 1,
  'name': 'bert_input_masks',
  'quantization': (0.0, 0),
  'quantization_parameters': {'quantized_dimension': 0,
   'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32)},
  'shape': array([  1, 500], dtype=int32),
  'shape_signature': array([  1, 500], dtype=int32),
  'sparsity_parameters': {}}]

In [11]:
sample_input_index = np.expand_dims(val_features_ids[0], 0).astype(np.int32)
sample_masks_index = np.expand_dims(val_features_masks[0], 0).astype(np.int32)

input_tensors = [sample_input_index, sample_masks_index]

In [12]:
bert_input_index = interpreter.get_input_details()[0]["index"]
bert_input_masks_index = interpreter.get_input_details()[1]["index"]
output_index = interpreter.get_output_details()[0]["index"]

interpreter.set_tensor(bert_input_index, sample_input_index)
interpreter.set_tensor(bert_input_masks_index, sample_masks_index)
interpreter.invoke()
prediction = interpreter.tensor(output_index)
print(prediction()[0]) # if prediction > 0.5 then class = 1 else class = 0

[0.9999035]


In [13]:
# Let's convert the data type of the validation features from int64 to int32
val_features_ids = val_features_ids.astype(np.int32)
val_features_masks = val_features_masks.astype(np.int32)

# Create TensorFlow dataset
val_ds = (
    tf.data.Dataset.from_tensor_slices((val_features_ids, val_features_masks, valid_sentiments))
    .batch(1)
    .cache()
    .prefetch(tf.data.experimental.AUTOTUNE)
)

In [14]:
def get_predictions(interpreter):
    predictions = []
    bert_input_index = interpreter.get_input_details()[0]["index"]
    bert_input_masks_index = interpreter.get_input_details()[1]["index"]
    output_index = interpreter.get_output_details()[0]["index"]

    for (i, features) in enumerate(val_ds):
        log_steps = 100
        tf.compat.v1.logging.log_every_n(tf.compat.v1.logging.INFO,
                                       'Processing example: #%d', log_steps, i)
        # Run inference
        interpreter.set_tensor(bert_input_index, features[0])
        interpreter.set_tensor(bert_input_masks_index, features[1])
        interpreter.invoke()

        # Parse the prediction
        prediction = interpreter.tensor(output_index)()[0]
        if prediction > 0.5:
            prediction = 1
        else:
            prediction = 0
        
        predictions.append(prediction)

    return predictions

In [15]:
interpreter = tf.lite.Interpreter(model_path="distilbert_sst_tflite.tflite")
interpreter.allocate_tensors()
predictions = get_predictions(interpreter)
accuracy = accuracy_score(valid_sentiments, predictions)
print("Accuracy: {}%".format(accuracy * 100.))

INFO:tensorflow:Processing example: #0
INFO:tensorflow:Processing example: #100
INFO:tensorflow:Processing example: #200
INFO:tensorflow:Processing example: #300
INFO:tensorflow:Processing example: #400
INFO:tensorflow:Processing example: #500
INFO:tensorflow:Processing example: #600
INFO:tensorflow:Processing example: #700
INFO:tensorflow:Processing example: #800
Accuracy: 90.36697247706422%


In [16]:
interpreter = tf.lite.Interpreter(model_path="distilbert_sst_tflite_fp16.tflite")
interpreter.allocate_tensors()
predictions = get_predictions(interpreter)
accuracy = accuracy_score(valid_sentiments, predictions)
print("Accuracy: {}%".format(accuracy * 100.))

INFO:tensorflow:Processing example: #0
INFO:tensorflow:Processing example: #100
INFO:tensorflow:Processing example: #200
INFO:tensorflow:Processing example: #300
INFO:tensorflow:Processing example: #400
INFO:tensorflow:Processing example: #500
INFO:tensorflow:Processing example: #600
INFO:tensorflow:Processing example: #700
INFO:tensorflow:Processing example: #800
Accuracy: 49.08256880733945%
